Csv and Excel file structure parsing

In [3]:
import pandas as pd
import os
#from langchain_community.document_loaders import CSVLoader, ExcelLoader

In [5]:
os.makedirs("data/structured_files", exist_ok=True)

In [ ]:
## Sample dataframe creation and saving as CSV
data = {
    "Name": ["John Doe", "Jane Smith", "Michael Brown", "Emily Johnson", "David Lee"],
    "Age": [28, 32, 45, 26, 34],
    "Gender": ["Male", "Female", "Male", "Female", "Male"],
    "Occupation": [
        "Software Engineer",
        "Data Analyst",
        "Project Manager",
        "UI/UX Designer",
        "DevOps Engineer"
    ],
    "Country": ["USA", "Canada", "UK", "Germany", "Australia"],
    "Email": [
        "johndoe@example.com",
        "janesmith@example.com",
        "michaelbrown@example.com",
        "emilyjohnson@example.com",
        "davidlee@example.com"
    ]
}

df = pd.DataFrame(data)
df.to_csv("data/structured_files/products.csv", index=False)

In [10]:
# saving as Excel with multiple sheets
with pd.ExcelWriter("data/structured_files/inventory.xlsx") as writer:
    df.to_excel(writer, index=False, sheet_name="Products")

    #Another Sheet
    data_another = {
        "category": ["Electronics", "Clothing"],
        "total_items": [150, 200],
        "total_value": [50000, 30000]
    }

    pd.DataFrame(data_another).to_excel(writer, index=False, sheet_name="Summary")

Csv Processing

In [11]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

In [17]:
### CSVloader parsing
csv_loader = CSVLoader(
    file_path = "data/structured_files/products.csv",
    encoding="utf-8",
    csv_args={"delimiter": ",",
              "quotechar": '"'}
)

csv_doc = csv_loader.load()

#print(csv_doc)
print(f"Number of documents loaded from CSV: {len(csv_doc)}")
print(f"content:\n{csv_doc[0].page_content}")
print(f"metadata:\n{csv_doc[0].metadata}")

Number of documents loaded from CSV: 5
content:
Name: John Doe
Age: 28
Gender: Male
Occupation: Software Engineer
Country: USA
Email: johndoe@example.com
metadata:
{'source': 'data/structured_files/products.csv', 'row': 0}


In [37]:
from typing import List
from langchain_core.documents import Document

### Creating a custom csv rich metadata parser
def process_csv(file_path: str)->List[Document]:
    df = pd.read_csv(file_path)
    docs = []

    for index, row in df.iterrows():
        content=f"""Personal Information:
            Name: {row['Name']},
            Age: {row['Age']},
            Gender: {row['Gender']},
            Occupation: {row['Occupation']},
            Country: {row['Country']},
            Email: {row['Email']}"""

## create Document with rich metadata
        doc = Document(
            page_content=content,
            metadata={
                "source": file_path,
                "row_index": index,
                "person": row['Name'],
                "occupation": row['Occupation'],
                "data_type":"personal_info"
            }
        )
        docs.append(doc)
    return docs

In [38]:
documents = process_csv("data/structured_files/products.csv")
print(f"Number of documents created with rich metadata: {len(documents)}")
print(f"content:\n{documents[3].page_content}")
print(f"metadata:\n{documents[3].metadata}")

Number of documents created with rich metadata: 5
content:
Personal Information:
            Name: Emily Johnson,
            Age: 26,
            Gender: Female,
            Occupation: UI/UX Designer,
            Country: Germany,
            Email: emilyjohnson@example.com
metadata:
{'source': 'data/structured_files/products.csv', 'row_index': 3, 'person': 'Emily Johnson', 'occupation': 'UI/UX Designer', 'data_type': 'personal_info'}


Excel processing

In [49]:
from typing import List
from langchain_core.documents import Document

def process_excel(file_path: str)->List[Document]:
    excel_file = pd.ExcelFile(file_path)
    docs=[]

    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        sheet_content = f"sheet: {sheet_name}\n"
        sheet_content += f"columns: {", ".join(df.columns)}\n"
        sheet_content += f"row: {len(df)}\n\n"

        doc = Document(
            page_content=sheet_content,
            metadata={
                'source': file_path,
                'sheet_name': sheet_name,
                'num_columns': len(df.columns),
                'num_rows': len(df),
                'data_type': 'excel_sheet'
         }
        )
        docs.append(doc)
    
    return docs


In [51]:
documents = process_excel("data/structured_files/inventory.xlsx")
print(f"Number of documents created from Excel: {len(documents)}")
for doc in documents:
    print("-----")
    print(f"content:\n{doc.page_content}")
    print(f"metadata:\n{doc.metadata}")


Number of documents created from Excel: 2
-----
content:
sheet: Products
columns: Name, Age, Gender, Occupation, Country, Email
row: 5


metadata:
{'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Products', 'num_columns': 6, 'num_rows': 5, 'data_type': 'excel_sheet'}
-----
content:
sheet: Summary
columns: category, total_items, total_value
row: 2


metadata:
{'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Summary', 'num_columns': 3, 'num_rows': 2, 'data_type': 'excel_sheet'}
